In [1]:
# joepareti54@gmail.com  May 13 2022
#
# this model implements one more linear layer than test_pytorch_bin_class.ipynb
# however the performance is approximately the same
#
# binary classification : cancer, no cancer 
# using a dataset in scikit learn
#
# the model class definition is similar to the linear regression
# but the last layer is a sigmoid function


In [2]:
import numpy as np
import torch
from torch import nn, optim
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#

In [3]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [4]:
print('input shape ',X.shape)
print('label shape ',y.shape)
n_samples = X.shape[0]
n_features = X.shape[1]
print('n_samples ', n_samples)
print('n_features ', n_features)

input shape  (569, 30)
label shape  (569,)
n_samples  569
n_features  30


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('input shape trn ',X_train.shape)
print('input shape tst ',X_test.shape)

input shape trn  (455, 30)
input shape tst  (114, 30)


In [6]:
type(X_train)
type(X_test)
type(y_train)
type(y_test)

numpy.ndarray

In [7]:
# apply scaling
scaler = StandardScaler().fit(X_train)
X_TRAIN = scaler.transform(X_train)
X_TEST  = scaler.transform(X_test)

In [8]:
#convert data to pytorch tensors
#XX_train = torch.from_numpy(X_TRAIN, dtype=np.float32)
XX_train = torch.from_numpy(X_TRAIN.astype(np.float32))
XX_test  = torch.from_numpy(X_TEST.astype(np.float32))
print('shape of train tensor ',XX_train.shape)
print('shape of test tensor ',XX_test.shape)
YY_train = torch.from_numpy(y_train.astype(np.float32))
YY_test = torch.from_numpy(y_test.astype(np.float32))

shape of train tensor  torch.Size([455, 30])
shape of test tensor  torch.Size([114, 30])


In [9]:
# in order to understand what input size and output size need to be for this case:
# https://stackoverflow.com/questions/54916135/what-is-the-class-definition-of-nn-linear-in-pytorch
# 
class BinaryClassification(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.fc1 = nn.Linear(input_size, 15)
        self.fc2 = nn.Linear(15, output_size)
#        self.fc3 = nn.Linear(3,output_size)
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        
        
        y_pred = torch.sigmoid(x)
        return y_pred
        

In [10]:
model = BinaryClassification(XX_train.shape[1], 1)
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100000
epochs =  50000

In [11]:
yy_train = YY_train.view(-1,1)
yy_test  = YY_test.view(-1, 1)

In [12]:
for i in range(epochs):
    optimizer.zero_grad()
    output = model(XX_train)
    LOSS = loss(output, yy_train)
#    print(LOSS)
    LOSS.backward()
    optimizer.step()
    if i % 10000 == 0:
        print('epoch', i, 'loss', LOSS.item())

epoch 0 loss 0.30074024200439453
epoch 10000 loss 0.0022003361955285072
epoch 20000 loss 0.0021978130098432302
epoch 30000 loss 0.002197802299633622
epoch 40000 loss 0.002197802299633622


In [13]:
#test model
y_hat = model(XX_test)
for i in range(50):
    print(y_hat[i].item(),yy_test[i].item()  )

0.9999752044677734 1.0
5.779271186777335e-20 0.0
3.831067339993104e-15 0.0
1.0 1.0
1.0 1.0
0.0 0.0
1.3774996460774802e-31 0.0
0.06343021988868713 0.0
0.9994199275970459 1.0
1.0 1.0
0.9833664894104004 1.0
2.0074070405939892e-08 0.0
1.0 1.0
1.1194713211981622e-10 0.0
1.0 1.0
1.44159098451181e-19 0.0
1.0 1.0
1.0 1.0
1.0 1.0
4.898788750529485e-25 0.0
0.999998927116394 0.0
1.0 1.0
6.514632883638904e-36 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
7.418999877653011e-21 0.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.3597920012209386e-10 0.0
1.0 1.0
1.0700077006675126e-21 0.0
1.0 1.0
1.0 1.0
1.573284496981211e-18 0.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0014793869340792298 1.0
1.0 1.0
1.0 1.0
0.9629786610603333 1.0
1.0 1.0


In [14]:
# compute accuracy
with torch.no_grad():
    
    a = y_hat.round()
    A = (a == yy_test)
    acc = A.sum() / float(YY_test.shape[0])
    print(acc)


tensor(0.9649)
